In [2]:
#We have to take a file like 'week-12-complete.csv' from the load-upcoming-matches notebook and add odds.  It ends up like
#'owl-with-odds-week-12.csv'.  Now just load and predict!

In [31]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
np.set_printoptions(suppress=True)  #Suppress Scientific Notation

In [4]:
df_upcoming = pd.read_csv('owl-with-odds-week-12.csv')

In [5]:
df_upcoming.shape

(10, 58)

In [6]:
%run get-ev-scratch-paper.ipynb

,Unnamed: 0,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,0,459.0,-833.0,0.285714,0.714286,1.0
1,1,-110.0,-110.0,0.500000,0.500000,0.0
2,2,-385.0,299.0,0.416667,0.583333,1.0
3,3,-137.0,112.0,0.416667,0.583333,0.0
4,4,-139.0,106.0,0.779412,0.220588,1.0
...,...,...,...,...,...,...
90,90,135.0,-179.0,0.176471,0.823529,1.0
91,91,-323.0,252.0,0.779412,0.220588,0.0
92,92,-227.0,170.0,0.779412,0.220588,0.0
93,93,-141.0,106.0,0.833333,0.166667,1.0



          Number of matches: 95
          Number of bets: 81
          Number of winning bets: 44
          Number of losing bets: 37
          Number of underdog bets: 26
          Number of underdog wins: 8
          Number of underdog losses: 18
          Number of Favorite bets: 55
          Number of favorite wins: 36
          Number of favorite losses: 19
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -438.9048172864855
          Profit per bet: -5.418577991191179
          Profit per match: -4.620050708278795
          
          


In [7]:
df=pd.read_csv("owl-with-odds.csv")

In [8]:
#We need to create the ensemble we agreed on

In [9]:
subset = ['t1_odds', 't2_odds']
print(len(df))
df.dropna(subset=subset ,inplace=True)
print(len(df))

df['t1_odds'] = pd.to_numeric(df['t1_odds'], errors='coerce')
df['t2_odds'] = pd.to_numeric(df['t2_odds'], errors='coerce')
df.dropna(subset=subset ,inplace=True)
print(len(df))

651
538
536


In [16]:
df_train = df.copy() #Don't need to split since we are predicting...
df_test = df_upcoming.copy()

In [14]:
#inclusive features
features = ['corona_virus_isolation', 't1_wins_season',
       't1_losses_season', 't2_wins_season', 't2_losses_season',
       't1_matches_season', 't2_matches_season', 't1_win_percent_season',
       't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime',
       't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime',
       't2_matches_alltime', 't1_win_percent_alltime',
       't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3',
       't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3',
       't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5',
       't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5',
       't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10',
       't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10',
       't2_win_percent_last_10', 
        't1_wins_vs_t2', 't1_losses_vs_t2',
       't1_matches_vs_t2', 't1_odds', 't2_odds', 'winner_label']

In [17]:
df_train_filtered = df_train[features].copy()
df_test_filtered = df_test[features].copy()

In [18]:
df_test_filtered.dropna(inplace=True)
df_train_filtered.dropna(inplace=True)

print(df_test_filtered.shape)
print(df_train_filtered.shape)

(10, 41)
(439, 41)


In [21]:
#model 3
model_3 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')

features_3 = ['t1_wins_last_3', 't2_matches_alltime', 'winner_label', 't1_odds', 't2_odds']

X=df_train_filtered[features_3].copy()
X_test=df_test_filtered[features_3].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_3.fit(X, y)
probs=model_3.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)
probs_3 = probs


          Number of matches: 10
          Number of bets: 9
          Number of winning bets: 5
          Number of losing bets: 4
          Number of underdog bets: 3
          Number of underdog wins: 2
          Number of underdog losses: 1
          Number of Favorite bets: 6
          Number of favorite wins: 3
          Number of favorite losses: 3
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: 161.25603864734302
          Profit per bet: 17.917337627482556
          Profit per match: 16.125603864734302
          
          


In [23]:
#model_5
#model_5 

model_5 = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.2, loss='deviance', max_depth=16,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=6, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=75, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

features_5 = ['corona_virus_isolation', 't1_wins_season', 't1_losses_season', 't2_wins_season', 't2_losses_season', 't1_matches_season', 't2_matches_season', 't1_win_percent_season', 't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime', 't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime', 't2_matches_alltime', 't1_win_percent_alltime', 't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3', 't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3', 't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5', 't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5', 't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10', 't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10', 't2_win_percent_last_10', 't1_wins_vs_t2', 't1_losses_vs_t2', 't1_matches_vs_t2', 'winner_label', 't1_odds', 't2_odds']

X=df_train_filtered[features_5].copy()
X_test=df_test_filtered[features_5].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_5.fit(X, y)
probs=model_5.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)
probs_5 = probs


          Number of matches: 10
          Number of bets: 10
          Number of winning bets: 2
          Number of losing bets: 8
          Number of underdog bets: 2
          Number of underdog wins: 0
          Number of underdog losses: 2
          Number of Favorite bets: 8
          Number of favorite wins: 2
          Number of favorite losses: 6
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -752.2222222222222
          Profit per bet: -75.22222222222221
          Profit per match: -75.22222222222221
          
          


In [32]:
print(probs_3)

[[0.57142857 0.42857143]
 [0.16666667 0.83333333]
 [0.75       0.25      ]
 [0.24137931 0.75862069]
 [1.         0.        ]
 [1.         0.        ]
 [0.33333333 0.66666667]
 [0.         1.        ]
 [1.         0.        ]
 [0.75       0.25      ]]


In [33]:
print(probs_5)

[[0.95543857 0.04456143]
 [0.03726252 0.96273748]
 [0.99947251 0.00052749]
 [0.88466574 0.11533426]
 [0.99985943 0.00014057]
 [0.9952812  0.0047188 ]
 [0.99822466 0.00177534]
 [0.07216555 0.92783445]
 [0.99893982 0.00106018]
 [0.99393876 0.00606124]]


In [34]:
ensemble_probs=(probs_3 + probs_5) * .5

In [35]:
print(ensemble_probs)

[[0.76343357 0.23656643]
 [0.10196459 0.89803541]
 [0.87473626 0.12526374]
 [0.56302253 0.43697747]
 [0.99992972 0.00007028]
 [0.9976406  0.0023594 ]
 [0.665779   0.334221  ]
 [0.03608278 0.96391722]
 [0.99946991 0.00053009]
 [0.87196938 0.12803062]]


In [69]:
def print_upcoming_predictions(df, probs):
    count=0
    bet_list = []
    for index, row in df.iterrows():
        t1 = row['team_one']
        t2=row['team_two']
        t1_prob = probs[count][0]
        t2_prob = probs[count][1]
        t1_bet_ev = get_bet_ev(row['t1_odds'], t1_prob)
        t2_bet_ev = get_bet_ev(row['t2_odds'], t2_prob)
        
        if t1_bet_ev > 0:
            bet_list.append(f"{t1} at {row['t1_odds']} has EV of {t1_bet_ev}")
        if t2_bet_ev > 0:
            bet_list.append(f"{t2} at {row['t2_odds']} has EV of {t2_bet_ev}")
        
        
        print(f"{t1} ({t1_prob:.2f}) vs {t2} ({t2_prob:.2f})")
        print(f"{t1} has odds of {row['t1_odds']}")
        print(f"{t1} has a bet EV of {t1_bet_ev:.2f}")
        print(f"{t2} has odds of {row['t2_odds']}")
        print(f"{t2} has a bet EV of {t2_bet_ev:.2f}")
        
        print()
        count=count+1
    return(bet_list)

In [70]:
bet_list = print_upcoming_predictions(df_upcoming, ensemble_probs)

Seoul Dynasty (0.76) vs Hangzhou Spark (0.24)
Seoul Dynasty has odds of -450
Seoul Dynasty has a bet EV of -6.69
Hangzhou Spark has odds of 360
Hangzhou Spark has a bet EV of 8.82

Guangzhou Charge (0.10) vs Shanghai Dragons (0.90)
Guangzhou Charge has odds of 400
Guangzhou Charge has a bet EV of -49.02
Shanghai Dragons has odds of -500
Shanghai Dragons has a bet EV of 7.76

New York Excelsior (0.87) vs Chengdu Hunters (0.13)
New York Excelsior has odds of -330
New York Excelsior has a bet EV of 13.98
Chengdu Hunters has odds of 270
Chengdu Hunters has a bet EV of -53.65

Atlanta Reign (0.56) vs Philadelphia Fusion (0.44)
Atlanta Reign has odds of 190
Atlanta Reign has a bet EV of 63.28
Philadelphia Fusion has odds of -230
Philadelphia Fusion has a bet EV of -37.30

San Francisco Shock (1.00) vs Los Angeles Valiant (0.00)
San Francisco Shock has odds of -680
San Francisco Shock has a bet EV of 14.70
Los Angeles Valiant has odds of 490
Los Angeles Valiant has a bet EV of -99.96

Hangzho

In [72]:
for b in bet_list:
    print(b)

Hangzhou Spark at 360 has EV of 8.820556353960356
Shanghai Dragons at -500 has EV of 7.764249002655324
New York Excelsior at -330 has EV of 13.98078487156368
Atlanta Reign at 190 has EV of 63.276532663804986
San Francisco Shock at -680 has EV of 14.697820485850066
Hangzhou Spark at -190 has EV of 52.27145993427966
Shanghai Dragons at -130 has EV of 17.791668537885855
New York Excelsior at -360 has EV of 23.167200827490245
Florida Mayhem at -210 has EV of 47.54079644130842
Houston Outlaws at 120 has EV of 91.83326387124168
